In [1]:
# First cell - Imports
%load_ext autoreload
%autoreload 2

# Python Libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display
from plotly.subplots import make_subplots
from scipy import stats


# Modules
from fred_loader import fred_load
from fred_transformer import fred_transform
from fred_visualizer import fred_visualize, fred_export

In [2]:
# Cell 2: Load Data
df = fred_load()
print("Data loaded successfully!")
#Verify data loaded
display(df)

Data loaded successfully!


,yield_spread,gdp,fed_funds,unemployment,option_adjusted_spread,delinquency_rate_credit_cards,delinquency_rate_loans,cpi,pce
date,,,,,,,,,
1976-01-01,NaN,6323.649,5.37,7.9,NaN,NaN,NaN,55.8,27.795
1976-01-02,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN
1976-01-03,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN
1976-01-04,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN
1976-01-05,NaN,NaN,5.29,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-11-11,NaN,NaN,4.58,NaN,2.63,NaN,NaN,NaN,NaN
2024-11-12,0.09,NaN,4.58,NaN,2.61,NaN,NaN,NaN,NaN
2024-11-13,0.17,NaN,4.58,NaN,2.64,NaN,NaN,NaN,NaN


In [3]:
# Cell 3: Transform Data
df = fred_transform(df, start_date='1996-12-31')
print("Transformation complete!")
display(df)
# Print to Excel
df.to_excel('stats.xlsx')

Transformation complete!


c:\Users\mrasm\econ_indicator_analysis\fred_transformer.py:9: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['gdp_growth'] = df['gdp'].pct_change(periods=4) * 100


,yield_spread,gdp,fed_funds,unemployment,option_adjusted_spread,delinquency_rate_credit_cards,delinquency_rate_loans,cpi,pce,gdp_growth,quarterly_spread,loan_delinq_3m_forward,loan_delinq_6m_forward,loan_delinq_9m_forward,loan_delinq_12m_forward,loan_delinq_18m_forward,loan_delinq_24m_forward,quarter,economic_period
date,,,,,,,,,,,,,,,,,,,
1996-12-31,0.55,NaN,6.26,NaN,3.13,4.70,NaN,NaN,NaN,0.000000,3.130000,1.90,1.81,1.72,1.70,1.73,1.87,4Q96,Pre-GFC (1996-2007)
1997-01-01,NaN,12115.472,6.26,5.3,3.06,4.70,1.9,159.4,70.005,0.645443,2.900778,1.81,1.72,1.70,1.81,1.77,1.97,1Q97,Pre-GFC (1996-2007)
1997-01-02,0.57,NaN,5.79,NaN,3.06,4.73,NaN,NaN,NaN,0.645443,2.900778,1.81,1.72,1.70,1.81,1.77,1.97,1Q97,Pre-GFC (1996-2007)
1997-01-03,0.57,NaN,5.17,NaN,3.09,4.73,NaN,NaN,NaN,0.645443,2.900778,1.81,1.72,1.70,1.81,1.77,1.97,1Q97,Pre-GFC (1996-2007)
1997-01-04,NaN,NaN,5.17,NaN,3.10,4.73,NaN,NaN,NaN,0.645443,2.900778,1.81,1.72,1.70,1.81,1.77,1.97,1Q97,Pre-GFC (1996-2007)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-11,NaN,NaN,4.58,NaN,2.63,NaN,NaN,NaN,NaN,0.000000,2.861778,NaN,NaN,NaN,NaN,NaN,NaN,4Q24,Covid to Present (2020-2024)
2024-11-12,0.09,NaN,4.58,NaN,2.61,NaN,NaN,NaN,NaN,0.000000,2.861778,NaN,NaN,NaN,NaN,NaN,NaN,4Q24,Covid to Present (2020-2024)
2024-11-13,0.17,NaN,4.58,NaN,2.64,NaN,NaN,NaN,NaN,0.000000,2.861778,NaN,NaN,NaN,NaN,NaN,NaN,4Q24,Covid to Present (2020-2024)


In [4]:
# Do all analysis and get plots
# Add this before calling fred_visualize
%load_ext autoreload
%autoreload 2
stats_table,current_plot, predictive_plot, time_series_plot = fred_visualize(df)

# Show results
stats_table.show()
current_plot.show()
predictive_plot.show()
time_series_plot.show()

fred_export(stats_table, current_plot, predictive_plot, time_series_plot)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Credit Risk Analysis Results:
